In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib

matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

In [2]:
datasets = {
    "fetal_head": {
        "name": "Fetal head",
        "path": "fetal_head.csv",
    },
    "fetal_head_2": {
        "name": "Fetal head 2",
        "path": "fetal_head_2.csv",
    },
    "kidneys": {
        "name": "Kidneys",
        "path": "kidneys.csv",
    },
    "lungs": {
        "name": "Lungs",
        "path": "lungs.csv",
    },
    "nerve": {
        "name": "Nerve",
        "path": "nerve.csv",
    },
    "thyroid": {
        "name": "Thyroid",
        "path": "thyroid.csv",
    },
    "shapes_a": {
        "name": "Shapes A",
        "path": "shapes_a.csv",
    },
    "shapes_b": {
        "name": "Shapes B",
        "path": "shapes_b.csv",
    },
}

repr_datasets = datasets.copy()
del repr_datasets["shapes_a"]
del repr_datasets["shapes_b"]
del repr_datasets["lungs"]

In [3]:
# mean of all representative datasets
df_mean = None
n_datasets = 0
for dataset in repr_datasets.keys():

    n_datasets += 1

    path = datasets[dataset]["path"]
    df = pd.read_csv(path, delimiter="\t").T.reset_index()
    df = df.rename(columns=df.iloc[0]).drop(df.index[0]).astype(float)
    df_np = df.to_numpy()
    if df_mean is None:
        df_mean = df_np
    else:
        df_mean += df_np

df_mean /= n_datasets
df_mean = pd.DataFrame(df_mean, columns=df.columns)
df_mean["trf_size"] = df_mean["trf_size"].astype(int)
df_mean

,trf_size,training_time,erf_rate_before_training,erf_rate,dice_score,object_rate,accuracy,sensitivity,specificity,jaccard_score
0,54,13.6,0.129154,0.587587,0.623221,30.518038,0.957659,0.721485,0.964616,0.504257
1,100,16.0,0.036007,0.389000,0.733017,7.629509,0.954901,0.810682,0.964196,0.624682
2,146,14.2,0.026212,0.408604,0.785382,3.390893,0.966713,0.833601,0.973246,0.689378
3,204,14.8,0.008689,0.178951,0.831666,1.827076,0.977702,0.854846,0.984900,0.749695
4,230,17.8,0.021866,0.247141,0.819782,1.220722,0.972406,0.852696,0.977477,0.733728
5,298,19.6,0.006683,0.130412,0.824968,0.812034,0.977081,0.871545,0.980200,0.745602
6,360,19.6,0.016344,0.205337,0.843211,0.498273,0.981117,0.871708,0.985485,0.769737
7,412,18.0,0.001777,0.093707,0.839186,0.447203,0.980994,0.875601,0.986093,0.763944
8,486,23.6,0.006308,0.175368,0.852429,0.292332,0.981978,0.873112,0.987972,0.777323
9,570,22.8,0.005208,0.090699,0.858223,0.203008,0.982667,0.874747,0.988190,0.784177


In [4]:
fig, axs = plt.subplots(2, 2, figsize=(12, 12))

plt.subplot(2, 2, 1)

for dataset in repr_datasets.keys():
    name = datasets[dataset]["name"]
    path = datasets[dataset]["path"]
    
    df = pd.read_csv(path, delimiter="\t").T.reset_index()
    df = df.rename(columns=df.iloc[0]).drop(df.index[0])
    df["trf_size"] = df["trf_size"].astype(int)
    plt.plot(df["trf_size"], df["dice_score"], label=name, linestyle="dotted")
    
plt.plot(df_mean["trf_size"], df_mean["dice_score"], label="Mean", color="black", linestyle="solid")
plt.ylim(0.46, 1.01)
plt.xlim(54, 570)
plt.xlabel("TRF Size")
plt.ylabel("Dice Score")
plt.title("(a) TRF Size vs Dice Score")
plt.legend()

plt.subplot(2, 2, 2)

for dataset in repr_datasets.keys():
    name = datasets[dataset]["name"]
    path = datasets[dataset]["path"]
    
    df = pd.read_csv(path, delimiter="\t").T.reset_index()
    df = df.rename(columns=df.iloc[0]).drop(df.index[0])
    df["trf_size"] = df["trf_size"].astype(int)
    # print(min(df["dice_score"]))
    plt.plot(df["trf_size"], df["erf_rate"], label=name, linestyle="dotted")
    
plt.plot(df_mean["trf_size"], df_mean["erf_rate"], label="Mean", color="black", linestyle="solid")
# plt.ylim(0.46, 1.01)
plt.xlim(54, 570)
plt.xlabel("TRF Size")
plt.ylabel("ERF Rate")
plt.title("(b) TRF Size vs ERF Rate")
plt.legend()

plt.subplot(2, 2, 3)

for dataset in repr_datasets.keys():
    name = datasets[dataset]["name"]
    path = datasets[dataset]["path"]
    
    df = pd.read_csv(path, delimiter="\t").T.reset_index()
    df = df.rename(columns=df.iloc[0]).drop(df.index[0])
    df["trf_size"] = df["trf_size"].astype(int)
    print(min(df["training_time"]), max(df["training_time"]))
    plt.plot(df["trf_size"], df["training_time"], label=name, linestyle="dotted")
    
plt.plot(df_mean["trf_size"], df_mean["training_time"], label="Mean", color="black", linestyle="solid")
plt.xlim(54, 570)
plt.xlabel("TRF Size")
plt.ylabel("Training Time (epochs)")
plt.title("(c) TRF Size vs Training Time")
plt.legend()

ax1 = plt.subplot(2, 2, 4)
#  = plt.subplots(figsize=(5, 4))

ax2 = ax1.twinx()

ax1.plot(df_mean["trf_size"], df_mean["dice_score"], label="Dice score")
ax1.plot(df_mean["trf_size"], df_mean["erf_rate"], label="ERF rate")
ax2.plot(df_mean["trf_size"], df_mean["training_time"], label="Train time", color="green")


ax1.set_ylabel("Score or Rate")
ax2.set_ylabel("Epochs")


lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines + lines2, labels + labels2, loc="best")

ax1.set_xlabel("TRF size")
plt.title("(d) Mean Results")
ax1.set_xlim(54, 570)

plt.show()
# plt.savefig("analysis.pgf")

21.0 38.0
1.0 25.0
21.0 54.0
7.0 17.0
1.0 7.0


/tmp/ipykernel_469/1488375060.py:83: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [5]:
attention_datasets = {
    "fetal_head": {
        "name": "Fetal head",
        "path": "attention/fetal_head.csv",
    },
    "fetal_head_2": {
        "name": "Fetal head 2",
        "path": "attention/fetal_head_2.csv",
    },
    "kidneys": {
        "name": "Kidneys",
        "path": "attention/kidneys.csv",
    },
    "lungs": {
        "name": "Lungs",
        "path": "attention/lungs.csv",
    },
    "nerve": {
        "name": "Nerve",
        "path": "attention/nerve.csv",
    },
    "thyroid": {
        "name": "Thyroid",
        "path": "attention/thyroid.csv",
    }
}

repr_datasets_attention = attention_datasets.copy()
del repr_datasets_attention["lungs"]

# mean of all representative datasets
df_mean_attention = None
n_datasets = 0
for dataset in repr_datasets_attention.keys():

    n_datasets += 1

    path = attention_datasets[dataset]["path"]
    df = pd.read_csv(path, delimiter="\t").T.reset_index()
    df = df.rename(columns=df.iloc[0]).drop(df.index[0]).astype(float)
    df_np = df.to_numpy()
    if df_mean_attention is None:
        df_mean_attention = df_np
    else:
        df_mean_attention += df_np

df_mean_attention /= n_datasets
df_mean_attention = pd.DataFrame(df_mean_attention, columns=df.columns)
df_mean_attention["trf_size"] = df_mean_attention["trf_size"].astype(int)
df_mean_attention

,trf_size,training_time,erf_rate_before_training,erf_rate,dice_score,object_rate,accuracy,sensitivity,specificity,jaccard_score
0,54,16.2,0.85294,0.80474,0.66778,30.51806,0.94836,0.75480,0.95350,0.53884
1,100,14.8,0.52372,0.68378,0.79174,7.62950,0.96926,0.84204,0.97392,0.69382
2,146,13.8,0.41750,0.62520,0.80806,3.39088,0.97518,0.85630,0.98096,0.71728
3,204,20.0,0.48328,0.15048,0.83652,1.82706,0.97700,0.87426,0.98020,0.75014
4,230,21.0,0.33746,0.48892,0.84004,1.22070,0.97988,0.86386,0.98592,0.76014
5,298,22.0,0.39136,0.08030,0.84038,0.81202,0.98084,0.86360,0.98586,0.76162
6,360,23.4,0.30764,0.27312,0.84796,0.49826,0.98182,0.87412,0.98666,0.77392
7,412,21.6,0.45216,0.01890,0.85428,0.44720,0.98220,0.87986,0.98826,0.77786
8,486,23.6,0.31246,0.09730,0.85840,0.29234,0.98234,0.87352,0.98842,0.78366
9,570,23.4,0.29698,0.06722,0.85722,0.21272,0.97882,0.87798,0.98516,0.78178


In [6]:
fig, big_axes = plt.subplots( figsize=(11, 6) , nrows=2, ncols=1, sharey=True) 

for row, big_ax in enumerate(big_axes, start=1):
    # big_ax.set_title("Subplot row %s \n" % row)

    if row == 1:
        big_ax.set_title("(a) Regular U-Net")
    else:
        big_ax.set_title("(b) Attention U-Net")

    # Turn off axis lines and ticks of the big subplot 
    # obs alpha is 0 in RGBA string!
    big_ax.tick_params(labelcolor=(1.,1.,1., 0.0), top='off', bottom='off', left='off', right='off')
    # removes the white frame
    big_ax._frameon = False


for i in range(1,9):
    ax = fig.add_subplot(2,4,i)
    
    # trf vs dice score on regular unet
    if i == 1:
        for dataset in repr_datasets.keys():
            name = datasets[dataset]["name"]
            path = datasets[dataset]["path"]
            
            df = pd.read_csv(path, delimiter="\t").T.reset_index()
            df = df.rename(columns=df.iloc[0]).drop(df.index[0])
            df["trf_size"] = df["trf_size"].astype(int)
            plt.plot(df["trf_size"], df["dice_score"], label=name, linestyle="dotted")
            
        plt.plot(df_mean["trf_size"], df_mean["dice_score"], label="Mean", color="black", linestyle="solid")
        plt.ylim(0.46, 1.01)
        plt.xlim(0, 576)
        plt.xticks([0, 288, 576])
        plt.xlabel("TRF Size")
        plt.ylabel("Dice Score")
        # plt.title("(a) Regular U-Net")
        # plt.legend()

    # trf vs erf rate on regular unet
    elif i == 2:
        for dataset in repr_datasets.keys():
            name = datasets[dataset]["name"]
            path = datasets[dataset]["path"]
            
            df = pd.read_csv(path, delimiter="\t").T.reset_index()
            df = df.rename(columns=df.iloc[0]).drop(df.index[0])
            df["trf_size"] = df["trf_size"].astype(int)
            # print(min(df["dice_score"]))
            plt.plot(df["trf_size"], df["erf_rate"], label=name, linestyle="dotted")
            
        plt.plot(df_mean["trf_size"], df_mean["erf_rate"], label="Mean", color="black", linestyle="solid")
        # plt.ylim(0.46, 1.01)
        plt.xlim(0, 576)
        plt.xticks([0, 288, 576])
        plt.xlabel("TRF Size")
        plt.ylabel("ERF Rate")
        # plt.title("(b) TRF Size vs ERF Rate")
        # plt.legend()

    # trf vs training time on regular unet
    elif i == 3:
        for dataset in repr_datasets.keys():
            name = datasets[dataset]["name"]
            path = datasets[dataset]["path"]
            
            df = pd.read_csv(path, delimiter="\t").T.reset_index()
            df = df.rename(columns=df.iloc[0]).drop(df.index[0])
            df["trf_size"] = df["trf_size"].astype(int)
            print(min(df["training_time"]), max(df["training_time"]))
            plt.plot(df["trf_size"], df["training_time"], label=name, linestyle="dotted")
            
        plt.plot(df_mean["trf_size"], df_mean["training_time"], label="Mean", color="black", linestyle="solid")
        plt.xlim(0, 576)
        plt.xticks([0, 288, 576])
        plt.xlabel("TRF Size")
        plt.ylabel("Training Time (epochs)")
        # plt.title("(c) TRF Size vs Training Time")
        # plt.legend()

    # object rate vs dice score on regular unet
    elif i == 4:
        for dataset in repr_datasets.keys():
            name = datasets[dataset]["name"]
            path = datasets[dataset]["path"]
            
            df = pd.read_csv(path, delimiter="\t").T.reset_index()
            df = df.rename(columns=df.iloc[0]).drop(df.index[0])
            # df["trf_size"] = df["trf_size"].astype(int)
            # notmalize object rate
            
            plt.plot(df["object_rate"], df["dice_score"], label=name, linestyle="dotted")
            
        plt.plot(df_mean["object_rate"], df_mean["dice_score"], label="Mean", color="black", linestyle="solid")
        # plt.ylim(0.46, 1.01)
        # plt.xlim(54, 570)
        plt.xscale("log")
        plt.xlabel("Object Rate (log scale)")
        plt.ylabel("Dice Score")
        # plt.title("(a) TRF Size vs Dice Score")
        # vertical line at 1
        plt.axvline(x=1, color="gray", linestyle="dashed")

    # trf size vs dice score on attention unet
    if i == 5:
        for dataset in repr_datasets_attention.keys():
            name = attention_datasets[dataset]["name"]
            path = datasets[dataset]["path"]
            
            df = pd.read_csv(path, delimiter="\t").T.reset_index()
            df = df.rename(columns=df.iloc[0]).drop(df.index[0])
            df["trf_size"] = df["trf_size"].astype(int)
            plt.plot(df["trf_size"], df["dice_score"], label=name, linestyle="dotted")
            
        plt.plot(df_mean_attention["trf_size"], df_mean_attention["dice_score"], label="Mean", color="black", linestyle="solid")
        plt.ylim(0.46, 1.01)
        plt.xlim(0, 576)
        plt.xticks([0, 288, 576])
        plt.xlabel("TRF Size")
        plt.ylabel("Dice Score")

    # trf vs erf rate on attention unet
    elif i == 6:
        for dataset in repr_datasets_attention.keys():
            name = attention_datasets[dataset]["name"]
            path = datasets[dataset]["path"]
            
            df = pd.read_csv(path, delimiter="\t").T.reset_index()
            df = df.rename(columns=df.iloc[0]).drop(df.index[0])
            df["trf_size"] = df["trf_size"].astype(int)
            # print(min(df["dice_score"]))
            plt.plot(df["trf_size"], df["erf_rate"], label=name, linestyle="dotted")
            
        plt.plot(df_mean_attention["trf_size"], df_mean_attention["erf_rate"], label="Mean", color="black", linestyle="solid")
        # plt.ylim(0.46, 1.01)
        plt.xlim(0, 576)
        plt.xticks([0, 288, 576])
        plt.xlabel("TRF Size")
        plt.ylabel("ERF Rate")

    # trf vs training time on attention unet
    elif i == 7:
        for dataset in repr_datasets_attention.keys():
            name = attention_datasets[dataset]["name"]
            path = datasets[dataset]["path"]
            
            df = pd.read_csv(path, delimiter="\t").T.reset_index()
            df = df.rename(columns=df.iloc[0]).drop(df.index[0])
            df["trf_size"] = df["trf_size"].astype(int)
            print(min(df["training_time"]), max(df["training_time"]))
            plt.plot(df["trf_size"], df["training_time"], label=name, linestyle="dotted")
            
        plt.plot(df_mean_attention["trf_size"], df_mean_attention["training_time"], label="Mean", color="black", linestyle="solid")
        plt.xlim(0, 570)
        plt.xticks([0, 288, 576])
        plt.xlabel("TRF Size")
        plt.ylabel("Training Time (epochs)")

    # object rate vs dice score on attention unet
    elif i == 8:
        for dataset in repr_datasets_attention.keys():
            name = attention_datasets[dataset]["name"]
            path = datasets[dataset]["path"]
            
            df = pd.read_csv(path, delimiter="\t").T.reset_index()
            df = df.rename(columns=df.iloc[0]).drop(df.index[0])
            # df["trf_size"] = df["trf_size"].astype(int)
            # notmalize object rate
            
            plt.plot(df["object_rate"], df["dice_score"], label=name, linestyle="dotted")
            
        plt.plot(df_mean_attention["object_rate"], df_mean_attention["dice_score"], label="Mean", color="black", linestyle="solid")
        # plt.ylim(0.46, 1.01)
        # plt.xlim(54, 570)
        plt.xscale("log")
        plt.xlabel("Object Rate (log scale)")
        plt.ylabel("Dice Score")
        # vertical line at 1
        plt.axvline(x=1, color="gray", linestyle="dashed")        
    




fig.set_facecolor('w')
plt.tight_layout()
plt.legend(loc="upper center", bbox_to_anchor=(-1.55, -0.3), shadow=False, ncol=6, fontsize=13)
# plt.show()
plt.savefig("analysis.pgf",  bbox_inches = 'tight')

21.0 38.0
1.0 25.0
21.0 54.0
7.0 17.0
1.0 7.0
21.0 38.0
1.0 25.0
21.0 54.0
7.0 17.0
1.0 7.0
